In [57]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import math

In [58]:
class NeuralAccumlatorCell(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.W_hat = Parameter(torch.Tensor(in_dim,out_dim))
        self.M_hat = Parameter(torch.Tensor(in_dim,out_dim))
        self.W  = Parameter(torch.tanh(self.W_hat) * torch.sigmoid(self.M_hat))
        self.register_parameter('bias',None)
        init.kaiming_uniform_(self.W_hat,a=math.sqrt(5))
        init.kaiming_uniform_(self.M_hat,a=math.sqrt(5))
        
    def forward(self,input):
        return F.linear(input,self.W,self.bias)

In [59]:
nac = NeuralAccumlatorCell(1,1)
nac.forward(torch.Tensor([1]))

tensor([0.], grad_fn=<SqueezeBackward3>)

In [60]:
for name, param in nac.named_parameters():
    if param.requires_grad:
        print (name, param.size())

W_hat torch.Size([1, 1])
M_hat torch.Size([1, 1])
W torch.Size([1, 1])


In [61]:
class NAC(nn.Module):
    def __init__(self,num_layers,inp_nodes,hide_node,out_nodes):
        super().__init__()
        self.num_layers = num_layers
        self.inp_nodes = inp_nodes
        self.hide_node = hide_node
        self.out_nodes = out_nodes
        
        layers = []
        
        for i in range(num_layers):
            layers.append(NeuralAccumlatorCell(hide_node if i > 0 else inp_nodes,hide_node if i < num_layers - 1 else out_nodes))
            
        self.model = nn.Sequential(*layers)
        
    def forward(self,x):
        out = self.model(x)
        return out

In [62]:
z = NAC(3,3,2,1)

In [70]:
for name,param in z.named_parameters():
    if(param.requires_grad):
        print(name)

model.0.W_hat
model.0.M_hat
model.0.W
model.1.W_hat
model.1.M_hat
model.1.W
model.2.W_hat
model.2.M_hat
model.2.W


In [ ]:
tensor = to